# Cartographie avec d3

D3 contient un grand nombre de fonctions qui facilitent la création de cartes interactives. On peut également intégrer d3 avec Leaflet sans grand problème. Nous allons voir ici les bases de la cartographie thématique avec d3. Par contre, nous devons d'abord faire un petit détour par des aspects plus généraux d'optimisation de la cartographie interactive.

## 1. Faire mieux que du GeoJSON

Traditionnellement, des couches SIG vectorielles ont été stockées dans des fichiers Shape. Ce format n'étant pas très bien adapté à la cartographie sur le Web car il s'agit d'un format binaire qui en plus est réparti sur au moins 3 fichiers différents. Pour cette raison, le format GeoJson a vu le jour qui peut être considéré comme étant l'équivalent des fichiers Shape pour la cartographie sur le Web.

Un fichier GeoJSON a la structure suivante:

```json
{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": { "type": "Point", "coordinates": [36.706988, 55.664097] },
            "properties": { "name": "Khar Turan National Park" }
        },
        {
            "type": "Feature",
            "geometry": { "type": "Point", "coordinates": [37.547378, 56.381516] },
            "properties": { "name": "Golestan National Park" }
        }
    ]
}
```

Un fichier GeoJSON est facile à lire et à éditer, et avec QGIS ou des outils comme [ogr2ogr](http://www.gdal.org/ogr2ogr.html), il est facile d'en créer un à partir d'un fichier Shape p.ex.

Par contre, le problème principal des fichiers GeoJson est leur taille relativement importante même pour des régions relativement petites. C'est un problème lorsqu'on doit transférer ces fichiers par le Web. La taille importante des fichiers est dû à plusieurs raisons:

- Beaucoup d'éléments sont répétés, comme par exemple `"type": "Feature"`, `"geometry": { "type": "point", ... }`, ou encore les noms des attributs. L'exemple ci-dessus pourrait être écrit comme suit sans perdre de l'information:

        point                name
        36.706988,55.664097  Khar Turan National Park
        37.547378,56.381516  Golestan National Park
  
  ce qui fait 98 octets contre 459 octets, donc un _taux de compression_ de plus de 450%!
  

- La précision des coordonnées est souvent plus élevée que ce dont on a besoin pour la cartographie interactive. On n'a pas besoin d'avoir une précision au centimètre p.ex. On peut donc souvent enlever quelques décimales, style:

        point          name
        36.707,55.664  Khar Turan National Park
        37.547,56.382  Golestan National Park
  
  ce qui nous permet de réduire encore le volume de données (on appelle cette procédure __«quantization»__)


- Notamment pour les couches des polygones de limites administratives, très utilisées dans les cartes choroplèthes, quasiment chaque arrête de polygone est répétée. En effet, chaque unité administrative est définie par le polygone complet, et l'unité voisine répète la frontière commune. Ce format de données est appelé le __format spaghetti__, contrairement au __format topologique__ qui évite cette redondance. Le format topologique a par ailleurs l'avantage d'éviter des problèmes topologiques où la frontière commune n'est pas définie exactement de la même manière. Si nous changeons la façon de stocker nos géométries pour un format topologique, nous pouvons donc économiser presque la moitié des coordonnées...


- Beaucoup de couches vectorielles sont très peu généralisées et donc plus détaillées que nécessaire pour une cartographie thématique. En plus, le format spaghetti rend l'utilisation d'algorithmes automatiques de simplification difficile, en raison du risque d'introduire des erreurs topologiques. L'utilisation d'un format topologique facilite la __simplification__ automatique. Ceci permet également d'adapter le niveau de généralisation au niveau de zoom plus facilement.


- Dans certaines situations, nous n'avons pas besoin de l'entier du fichier des géometries, p.ex. si on visualise uniquement une partie de la région lors d'un zoom. Il suffirait dans ce cas de transmettre uniquement une sous-région du fichier initial.


- Les fichiers JSON transmis contiennent du texte ASCII, et contient souvent uniquement environ 60 à 70 caractères différents, comparé aux 256 caractères possibles. Une traduction en format binaire optimisé pourrait donc optimiser presque deux tiers du volume de données transmis.

Pour palier à ces différents problèmes, il y a deux approches qui ont vu le jour ces dernières années:

1. le [format TopoJson](https://github.com/topojson/topojson) de Mike Bostock, un des principaux développeurs de d3
2. les [Vector Tiles](https://www.mapbox.com/vector-tiles/) de Mapbox

Le format TopoJson adresse tous les points mentionnés ci-dessus sauf les deux derniers, tandis que les Vector Tiles focalisent plus sur ces deux derniers points. Nous allons couvrir ici uniquement le format TopoJson.


## 2. TopoJSON

TopoJSON est un format topologique qui compose un polygone à partir d'une séries de lignes (les _arcs_). Donc au lieu d'énumérer les points du polygone, il y a d'un côté une liste avec tous les arcs existants dans une ou plusieurs couches vectorielles, et de l'autre côté une liste d'arcs qui composent un polygone.

Un fichier TopoJson peut facilement intégrer plusieurs couches, comme p.ex. les couches des communes ensemble avec les districts et les cantons. Ceci fait du sens vu que les arcs à utiliser sont les mêmes.

Les formats topologiques ne sont pas nouveaux dans les SIG. En effet, les anciens logiciels SIG comme GRASS GIS et ARC/INFO avaient des formats topologiques pour les données vectorielles. ARC/INFO est devenu, après la version 7, ArcGIS 8, mais sans intégrer le format topologique, même si le mot _arc_ restait dans le nom...

En plus d'implémenter un format topologique, TopoJson intègre également quelques autres optimisations, comme la réduction du nombre des décimales. Le résultat est un format beaucoup plus compact que le GeoJSON, et qui se prête mieux pour d'autres optimisations, particulièrement au niveau de la généralisation.

Le blog de Mike Bostocks contient un excellent article sur le format TopoJSON et le raisonnement au niveau de la topologie qui est à la base. [[How To Infer Topology](https://bost.ocks.org/mike/topology/)]

`geo2topo` est un logiciel de commande de ligne développé par Mike Bostocks qui permet de traduire un ou plusieurs fichiers GeoJSON en format TopoJSON. Il peut être combiné avec d'autres logiciels de ligne de commande pour couvrir toute la chaîne de travail de la cartographie interactive (voir l'article _[«Command-Line Cartography»](https://medium.com/@mbostock/command-line-cartography-part-1-897aa8f8ca2c)_ sur le blog de Mike Bostocks).

Pour installer les logiciels nécessaires pour TopoJSON, le plus simple est de l'installer avec NPM:

```shell
npm install -g topojson
```

Si NPM n'est pas installé, il est également possible de [le télécharger](https://github.com/topojson/topojson/releases/latest). Ou mieux, on peut installer NPM (livré avec [Node JS](https://nodejs.org)), p.ex. avec Homebrew sur le Mac:

```shell
brew install node
```

Pour l'exercice, nous allons utiliser les couches des communes suisses, ensemble avec les lacs, disponibles en fichier GeoJSON:

- [Communes suisses 2017](https://raw.githubusercontent.com/christiankaiser/geovis2/master/data/communes-vec200-2017.geojson) (Vector-200, source Swisstopo, taille: 5.42 Mo)
- [Lacs suisses](https://raw.githubusercontent.com/christiankaiser/geovis2/master/data/vec200-lacs.geojson) (même source, taille: 284 Ko)

Notamment le fichier des communes est trop gros pour être chargé à travers le Web, et même compressé, il fait encore 1.2 Mo (la compression est généralement appliquée par défaut par les serveurs Web, sans que l'on remarque quelque chose).



La __conversion des fichiers GeoJSON en TopoJSON__ se fait pour toutes les couches à la fois. Ceci est important, car les lacs et les communes partagent les contours. La commande suivante permet de le faire:

```bash
geo2topo communes=communes-vec200-2017.geojson lacs=vec200-lacs.geojson > communes-lacs-topo.json
```

Ceci crée le fichier TopoJSON, qui a une taille de 2.5 Mo, sans avoir appliqué une simplification ou autre modification.

L'indication `communes=...` et `lacs=...` permet de donner un nom à chaque couche à l'intérieur du fichier TopoJSON. Nous pouvons choisir librement ce nom.

La taille du fichier après compression est encore environ 800 Ko, ce qui est déjà bien. Mais nous pouvons encore __simplifier les géométries__ un peu, nous n'avons pas besoin de tous les détails pour une carte à l'échelle de la Suisse. Ceci se fait avec la commande __toposimplify__:

```bash
toposimplify -p 100 -f < communes-lacs-topo.json > communes-lacs-simpl-topo.json
```

ce qui réduit la taille du fichier encore une fois un peu. Le paramètre `-p 100` élimine toutes les entités qui sont plus petites que 100 m<sup>2</sup>. `-f` élimine des éventuels arcs non utilisés.

On peut encore __réduire la précisions des coordonnées__:

```bash
topoquantize 1e4 < communes-lacs-simpl-topo.json > communes-lacs-quant-topo.json
```

Le paramètre de qualité est dans ce cas `1e4` (on peut aussi écrire `10000` à la place). Une valeur plus élevée indique une précision supérieure, mais des tailles de fichier plus grands.

Après cette opération, la taille du fichier GeoJSON est encore environ 1.2 Mo, ou 320 Ko après compression. Ce qui est déjà très acceptable...

__Astuce:__ on peut enchaîner les trois étapes en une seule commande de Terminal:

```bash
geo2topo communes=communes-vec200-2017.geojson lacs=vec200-lacs.geojson | toposimplify -p 100 -f | topoquantize 1e4 > communes-lacs-quant-topo.json
```

Ceci fonctionne grâce à l'opérateur `|` qui prend la sortie d'une commande comme entrée à la commande suivante.

On peut encore réduire la taille du fichier, en enlevant des attributs non utilisés, et en modifiant les noms des attributs (p.ex. au lieu de `POP_DENS`, utiliser `DNS`, ce qui permet d'enlever 5 caractères pour chacune des 2500 communes). Ainsi, on arrive à une taille d'environ 1 Mo pour le fichier TopoJSON, ou moins de 300 Ko après compression. À comparer avec les presque 5.5 Mo au départ!

Pour __contrôler le résultat de ces étapes__, il est possible de convertir le fichier TopoJSON à nouveau en fichier GeoJSON:

```bash
topo2geo communes=cmnes.geojson lacs=lacs.geojson < communes-lacs-quant-topo.json
```

ce qui crée les deux fichiers `cmnes.geojson` et `lacs.geojson`.

Pour l'instant, nous avons uniquement les limites des communes et les lacs. On peut encore rajouter les __limites des cantons__ par exemple, car nous avons un attribut qui associe chaque commune au canton. L'extraction fonctionne avec la commande __`topomerge`__:

```bash
topomerge -k 'd.properties.KANTONSNUM' cantons=communes < communes-lacs-quant-topo.json > communes-cantons-lacs-quant-topo.json
```

Le paramètre `-k 'd.properties.KANTONSNUM'` indique l'attribut qui doit être utilisé pour la fusion des géométries; `d` étant la commune (c'est défini par `topomerge`).

Notez aussi que la taille du fichier n'a quasiment pas changé, malgré le fait que nous avons ajouté tous les cantons!

Nous sommes maintenant prêts à faire une carte avec notre fichier TopoJSON.

## 3. Carte SVG

...